In [ ]:
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from PIL import Image
import pathlib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import keras
from keras import layers
from keras import layers
import keras
from keras.models import Sequential
import warnings
warnings.filterwarnings('ignore')
import csv

cmap = plt.get_cmap('inferno')
plt.figure(figsize=(8,8))
vowel = 'negative positive'.split()
for v in vowel:
    pathlib.Path(f'saved_image/{v}').mkdir(parents=True, exist_ok=True)
    for filename in os.listdir(f'./drive/MyDrive/vowel/{v}'):
        songname = f'./drive/MyDrive/vowel/{v}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=5)
        plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides='default', mode='default', scale='dB');
        plt.axis('off');
        plt.savefig(f'saved_image/{v}/{filename[:-3].replace(".", "")}.png')
        plt.clf()

header = 'filename chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 21):
    header += f' mfcc{i}'
header += ' label'
header = header.split()

file = open('vvowel_dataset.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)
vowel = 'negative positive'.split()
for v in vowel:
    for filename in os.listdir(f'./drive/MyDrive/vowel/{v}'):
        songname = f'./drive/MyDrive/vowel/{v}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=30)
        rmse=librosa.feature.rms(y=y)[0]
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'    
        for e in mfcc:
            to_append += f' {np.mean(e)}'
        to_append += f' {v}'
        file = open('vvowel_dataset.csv', 'a', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())


<Figure size 576x576 with 0 Axes>

In [ ]:

import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from PIL import Image
import pathlib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn import metrics
import keras
from keras import layers
from keras import layers
import keras
from keras.models import Sequential
from tensorflow.keras.layers import Dropout
import warnings
warnings.filterwarnings('ignore')
import csv
import tensorflow as tf
import time
from sklearn.datasets import make_classification
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score


data = pd.read_csv('/content/drive/MyDrive/vvowel_dataset.csv')
data.head()# Dropping unneccesary columns
data = data.drop(['filename'],axis=1)#Encoding the Labels
data_list = data.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(data_list)#Scaling the Feature columns
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))#Dividing data into training and Testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
start = time.time()
#create, train and save the model
model = Sequential()
model.add(layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.save('breath.model')

#Predit the audio and give the result
from sklearn.preprocessing import LabelEncoder
filename="/content/drive/MyDrive/positive_vowel.wav"
audio, sample_rate = librosa.load(filename, res_type='kaiser_fast') 
mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=26)
mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
mfccs_scaled_features=mfccs_scaled_features.reshape(1,-1)
predicted_label=model.predict_classes(mfccs_scaled_features)
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(['negative', 'positive'])
le.classes_
finaln = le.inverse_transform(predicted_label)


classifier = model.fit(X_train, y_train, epochs=100, batch_size=128)
test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print('Accuracy: ')
print(test_accuracy[1])
predictions = pd.Series(np.random.rand(y_train.shape[0]))
model = DummyClassifier(strategy='stratified')
model.fit(X_train, y_train)
yhat = model.predict_proba(X_test)
pos_probs = yhat[:, 1]
# calculate roc auc
roc_auc = roc_auc_score(y_test, pos_probs)
print('No Skill ROC AUC %.3f' % roc_auc)
# skilled model
model = LogisticRegression(solver='lbfgs')
model.fit(X_train, y_train)
yhat = model.predict_proba(X_test)
pos_probs = yhat[:, 1]
# calculate roc auc
roc_auc = roc_auc_score(y_test, pos_probs)
print('Logistic ROC AUC %.3f' % roc_auc)
stop = time.time()
print(f"Training time: {stop - start}s")


#print the result of prediction
print(mfccs_scaled_features)
print(mfccs_scaled_features)
print(mfccs_scaled_features.shape)
print(predicted_label)
print(finaln)

INFO:tensorflow:Assets written to: breath.model/assets
Epoch 1/100
1/1 [==============================] - 7s 7s/step - loss: 0.6957 - accuracy: 0.4186
Epoch 2/100
1/1 [==============================] - 0s 9ms/step - loss: 0.6351 - accuracy: 0.6977
Epoch 3/100
1/1 [==============================] - 0s 9ms/step - loss: 0.5898 - accuracy: 0.7209
Epoch 4/100
1/1 [==============================] - 0s 7ms/step - loss: 0.5598 - accuracy: 0.7209
Epoch 5/100
1/1 [==============================] - 0s 17ms/step - loss: 0.5327 - accuracy: 0.7209
Epoch 6/100
1/1 [==============================] - 0s 17ms/step - loss: 0.5061 - accuracy: 0.7209
Epoch 7/100
1/1 [==============================] - 0s 11ms/step - loss: 0.4804 - accuracy: 0.7209
Epoch 8/100
1/1 [==============================] - 0s 8ms/step - loss: 0.4548 - accuracy: 0.7209
Epoch 9/100
1/1 [==============================] - 0s 7ms/step - loss: 0.4296 - accuracy: 0.7442
Epoch 10/100
1/1 [==============================] - 0s 7ms/step - loss